In [1]:
import os
from pyspark.sql import SparkSession

# Configurations

In [2]:
minioConfigs = {
    "spark.hadoop.fs.s3a.impl":                "org.apache.hadoop.fs.s3a.S3AFileSystem",
    "spark.hadoop.fs.s3a.endpoint":            "http://minio:9000",
    "spark.hadoop.fs.s3a.access.key":          "admin",
    "spark.hadoop.fs.s3a.secret.key":          "SuperSecr3t",
    "spark.hadoop.fs.s3a.path.style.access":   True
}

In [3]:
packages = [
    # 'org.apache.hadoop:hadoop-aws:2.7.3'          # => static jars

    # 'com.amazonaws:aws-java-sdk-s3:1.11.234',     # API incompatible
    # 'com.amazonaws:aws-java-sdk-bundle:1.11.234', # 60M+, API incompatible
    # 'com.amazonaws:aws-java-sdk-bundle:1.7.4',    # no-such package
    # 'com.amazonaws:aws-java-sdk:1.7.4',           # not working

    #'org.apache.hive:hive-cli:1.2.1',
    #'org.apache.hive:hive-beeline:1.2.1',
    'org.apache.spark:spark-hive_2.11:2.4.4',
    'org.apache.spark:spark-hive-thriftserver_2.11:2.4.4'
]

excludes = [
    'com.fasterxml.jackson.core:jackson-annotations',
    'com.fasterxml.jackson.core:jackson-core',
    'com.fasterxml.jackson.core:jackson-databind',
    'com.fasterxml.jackson.dataformat:jackson-dataformat-cbor',
    'commons-cli:commons-cli',
    'commons-io:commons-io',
    'commons-lang:commons-lang',
    'commons-codec:commons-codec',
    'commons-logging:commons-logging',
    'joda-time:joda-time',
    'org.apache.httpcomponents:httpclient',
    'org.apache.httpcomponents:httpcore',
]

jars = [
    #'/home/jovyan/work/jars/*'
    #'/home/jovyan/.ivy2/jars/org.apache.hive*'
    '/home/jovyan/.ivy2/jars/*hive*'
]

In [4]:
from pyspark import SparkConf

conf = SparkConf()
conf.setAll(minioConfigs.items())
conf.set("spark.jars", ",".join(jars))
#conf.set("spark.jars.packages", ",".join(packages))
#conf.set("spark.jars.excludes", ",".join(excludes))

In [5]:
spark = SparkSession.builder\
        .appName("Teko::HiveQL")\
        .config(conf=conf)\
        .enableHiveSupport()\
        .getOrCreate()

# Read dataframe from MinIO

In [6]:
df = spark.read.parquet("s3a://dwh/actor")